In [1]:
#Author Lana Chen(m073040105@g-mail.nsysu.edu.tw)
#Update: Obt,10th,2019
#Transform news to word embedding, and store them in ./embedding/output.npy

In [2]:
import pandas as pd
import re
from shutil import copyfile
import numpy as np

In [3]:
#read file(It's important that the input file must look like the following: )
df_read= pd.read_csv('myinput.csv') #your news name
df_read

,context
0,〔記者林宥辰／綜合報導〕備戰台灣大賽，中信兄弟昨、今兩天都舉辦隊內分組賽，保持手感，週六首戰...


In [4]:
#Part1: throw(here as the instruction replace) the trash and split the sentences in each paragraphs.
#PS. You can define "trash" by yourself, e.g. for me, →●▲ are trash.
df_read = df_read.replace(" ","")
df_read = df_read.replace("→","")
df_read = df_read.replace("●","")
df_read = df_read.replace("▲","")
df_read = df_read.replace("◎","")
df_read = df_read.replace("","")
df_read = df_read.replace("","")
df_read = df_read.replace("★","")
df_read = df_read.replace("◆","")
df_read = df_read.replace("☆","")
df_read = df_read.replace("\n","")
df_read = re.split(r'[，。?!,.;；]',df_read['context'][0])
#display(df_read)

In [5]:
#Part2: do zero padding on words of each sentences no matter the number of it is smaller than 20 or not.
#Here we add 50 words each sentences, and you need to add more than 30 at least.
#(for the reason that tranformimg with BERT can cause problem as "0 0 0" to "000".)
for j in range( len( df_read ) ):
    s=''
    for k in range( len( df_read[j] ) ):
        s+= df_read[j][k]
    for m in range( 30 ):
        s+=' 0'
    df_read[j] = s

In [6]:
#Part4: do zero padding on paragraphs no matter whether it's number of sentences are smaller than 30.
sen_30zero='0'+' 0'*30
l=[]
for j in range( len( df_read ) ):
    l.append( df_read[j] )
for j in range( 30-len( df_read ) ):
    l.append( sen_30zero )
df_read = l

In [7]:
#Part5: to use BERT to do word embedding, we transfrom myinput.csv to myinput.txt line by line,
#transform myinput.txt to mytmpfile.jsonl,
#and at the end output my(word embedding about your news).

input_filename = 'myinput.txt' #your news(input file) name
output_filename = 'myoutput' #your output file name

bertfile =  open(input_filename, "w")
for i in range(len(df_read)):
    bertfile.writelines(str(df_read[i]))
    bertfile.writelines('\n')
bertfile.close()

#The word embedding information will start as INFO:tensorflow:*** Example ***(it must be seen!!!)
!python ./extract_features.py \
    --input_file=./myinput.txt \
    --output_file=./mytmpfile.jsonl \
    --vocab_file=./chinese_L-12_H-768_A-12/vocab.txt \
    --bert_config_file=./chinese_L-12_H-768_A-12/bert_config.json \
    --init_checkpoint=./chinese_L-12_H-768_A-12/bert_model.ckpt \
    --layers=-1 \
    --max_seq_length=20 \
    --batch_size=8

/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/ten

INFO:tensorflow:Calling model_fn.
I1014 21:26:32.686852 139857285744448 estimator.py:1145] Calling model_fn.
INFO:tensorflow:Running infer on CPU
I1014 21:26:32.687073 139857285744448 tpu_estimator.py:2965] Running infer on CPU

W1014 21:26:32.689415 139857285744448 deprecation_wrapper.py:119] From /home/k66/桌面/predict_news/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.


W1014 21:26:32.690595 139857285744448 deprecation_wrapper.py:119] From /home/k66/桌面/predict_news/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.


W1014 21:26:32.711364 139857285744448 deprecation_wrapper.py:119] From /home/k66/桌面/predict_news/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

Instructions for updating:
Use keras.layers.dense instead.
W1014 21:26:32.751507 139857285744448 deprecation.py:323] From /home/k66/桌面/predict_news/mode

W1014 21:26:33.397376 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a98dd898>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a98dd898>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1014 21:26:33.470229 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a98dd898>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

W1014 21:26:34.060282 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a9625ef0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a9625ef0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1014 21:26:34.146328 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a97f40f0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

W1014 21:26:34.786435 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a95ff358>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a95ff358>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1014 21:26:34.874926 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a92aaf60>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

...

W1014 21:26:38.176108 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a8add898>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a8add898>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1014 21:26:38.265964 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a8d59198>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

W1014 21:26:38.852042 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a93672e8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32a93672e8>>: AssertionError: Bad argument number for Name: 3, expecting 4
W1014 21:26:38.987256 139857285744448 ag_logging.py:145] Entity <bound method Dense.call of <tensorflow.python.layers.core.Dense object at 0x7f32aa4fff98>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Dense.call of <tensorflow.python.layers.core.Dense object 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1014 21:26:39.613376 139857285744448 deprecation.py:323] From /home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
I1014 21:26:39.849914 139857285744448 monitored_session.py:240] Graph was finalized.
2019-10-14 21:26:39.850225: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-10-14 21:26:39.873997: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3591655000 Hz
2019-10-14 21:26:39.874372: I tensorflow/compiler/xla/service/service.cc:168] XLA service

2019-10-14 21:26:40.381832: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: ./chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001; No such file or directory
2019-10-14 21:26:40.381848: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: ./chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001; No such file or directory
2019-10-14 21:26:40.382444: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: ./chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001; No such file or directory
2019-10-14 21:26:40.382734: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: ./chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001; No such file or directory
2019-10-14 21:26:40.383120: W tensorflow/core/framework/op_kernel.cc:1502] OP_REQUIRES failed at

  File "/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py", line 2709, in _call_model_fn
    config)
  File "/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/estimator.py", line 1146, in _call_model_fn
    model_fn_results = self._model_fn(features=features, **kwargs)
  File "/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py", line 2967, in _model_fn
    features, labels, is_export_mode=is_export_mode)
  File "/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py", line 1549, in call_without_tpu
    return self._call_model_fn(features, labels, is_export_mode=is_export_mode)
  File "/home/k66/anaconda3/envs/Lana/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py", line 1867, in _call_model_fn
    estimator_spec 

In [8]:
#Part6: Transform jsonl file to npy file.
df = pd.read_json('mytmpfile.jsonl',lines=True, orient='records')
a_sentence=[]
para=[]
for linex_index in range(30):
    a_sentence=[]
    for seq in range(20):
        a_sentence.append( df['features'][linex_index][seq]['layers'][0]['values'] )
    para.append(a_sentence)
np.save('embedding/'+output_filename+'.npy',para)
